In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import stats, optimize

def fit_t_regression(y, X):
    """
    MLE for Student-t regression with parameters (beta, sigma, nu).
    Model: y = X @ beta + e,  e ~ t_nu(loc=0, scale=sigma)
    X must already include an intercept column as the first column.
    Returns (beta, sigma, nu).
    """
    y = np.asarray(y, dtype=float).ravel()
    X = np.asarray(X, dtype=float)
    n, p = X.shape
    if y.size != n:
        raise ValueError("y and X have incompatible shapes.")
    if n < p + 1:
        raise ValueError("Not enough observations for regression.")

    # OLS init
    beta0, *_ = np.linalg.lstsq(X, y, rcond=None)
    resid0 = y - X @ beta0

    # Robust initial scale via MAD
    mad = np.median(np.abs(resid0 - np.median(resid0)))
    sigma0 = mad / 0.6744897501960817 if mad > 0 else np.std(resid0, ddof=p)
    if not np.isfinite(sigma0) or sigma0 <= 0:
        sigma0 = max(np.std(resid0, ddof=p), 1e-6)
    nu0 = 8.0

    # Reparameterize: sigma = exp(a), nu = exp(b) + 2  (sigma>0, nu>2)
    def pack(beta, sigma, nu):
        return np.concatenate([beta, [np.log(sigma), np.log(nu - 2.0)]])

    def unpack(theta):
        beta = theta[:p]
        sigma = np.exp(theta[p])
        nu = np.exp(theta[p + 1]) + 2.0
        return beta, sigma, nu

    def neg_loglik(theta):
        beta, sigma, nu = unpack(theta)
        if not (np.isfinite(sigma) and np.isfinite(nu) and sigma > 0 and nu > 2):
            return np.inf
        resid = y - X @ beta
        ll = stats.t.logpdf(resid, df=nu, loc=0.0, scale=sigma).sum()
        return -ll

    theta0 = pack(beta0, sigma0, nu0)
    res = optimize.minimize(
        neg_loglik,
        theta0,
        method="L-BFGS-B",
        options={"maxiter": 5000, "ftol": 1e-12}
    )
    if not res.success:
        raise RuntimeError(f"Optimization failed: {res.message}")

    beta_hat, sigma_hat, nu_hat = unpack(res.x)
    return beta_hat, float(sigma_hat), float(nu_hat)

# ------------------ Load data and run ------------------
DATA_DIR = Path.cwd() / "testfiles_" / "data"
df = pd.read_csv(DATA_DIR / "test7_3.csv")

# Select numeric columns
num = df.select_dtypes(include="number")
if num.shape[1] < 4:
    raise ValueError("CSV must contain at least four numeric columns (y + 3 predictors).")

# Prefer a 'y' column (case-insensitive). Otherwise, use the LAST numeric column as y.
lower_cols = [c.lower() for c in num.columns]
if "y" in lower_cols:
    y_col = num.columns[lower_cols.index("y")]
else:
    y_col = num.columns[-1]

# Predictors are the remaining numeric columns; take the first three in their original order.
X_cols = [c for c in num.columns if c != y_col][:3]

# Build arrays and drop non-finite rows
y = num[y_col].to_numpy(dtype=float)
X_raw = num[X_cols].to_numpy(dtype=float)
mask = np.isfinite(y) & np.all(np.isfinite(X_raw), axis=1)
y = y[mask]
X_raw = X_raw[mask]

# Add intercept
X = np.column_stack([np.ones(len(y)), X_raw])

beta, sigma, nu = fit_t_regression(y, X)

# Output in the required order
mu = 0.0
Alpha = beta[0]
B1, B2, B3 = beta[1], beta[2], beta[3]
print(f"mu:{mu:.1f},sigma:{sigma:.17f},nu:{nu:.17f},Alpha:{Alpha:.17f},B1:{B1:.17f},B2:{B2:.17f},B3:{B3:.17f}")


mu:0.0,sigma:0.04854806399547851,nu:4.59828246634698345,Alpha:0.04263419992591586,B1:0.97488831602876735,B2:2.04119125106135879,B3:3.15480204974339040
